In [1]:
import numpy as np
import pandas as pd
import json
from geopy.geocoders import Nominatim
from geopy.geocoders import Nominatim
import geocoder
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
print('Libraries imported.')

Libraries imported.


In [5]:
data = requests.get("https://en.wikipedia.org/wiki/Category:Suburbs_of_Mumbai").text
soup = BeautifulSoup(data, 'html.parser')
neighborhoodList = []
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)
kl_df = pd.DataFrame({"Neighborhood": neighborhoodList})
def get_latlng(neighborhood):
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Mumbai, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords
coords = [ get_latlng(neighborhood) for neighborhood in kl_df["Neighborhood"].tolist() ]
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
kl_df['Latitude'] = df_coords['Latitude']
kl_df['Longitude'] = df_coords['Longitude']
kl_df.head()


,Neighborhood,Latitude,Longitude
0,Andheri,19.118459,72.841763
1,Anushakti Nagar,19.042830,72.927340
2,Baiganwadi,19.062940,72.926630
3,Bandra,19.054370,72.840170
4,Bhandup,19.145560,72.948560


In [6]:
address = 'Mumbai, India'
geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Mumbai {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Mumbai 18.9387711, 72.8353355.


In [8]:
CLIENT_ID = 'J2HRL5UW01505JZ1TIF0T45AYZYHMXGEB2VXHRTOC3XT1OVV' # your Foursquare ID
CLIENT_SECRET = 'DOA1LVGX0RDHXLZOJPMEHRHRWR2Z25VKMLGMJ2MRSGMYE0QO' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: J2HRL5UW01505JZ1TIF0T45AYZYHMXGEB2VXHRTOC3XT1OVV
CLIENT_SECRET:DOA1LVGX0RDHXLZOJPMEHRHRWR2Z25VKMLGMJ2MRSGMYE0QO


In [9]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

venues_df = pd.DataFrame(venues)
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']
venues_df.head()

,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Andheri,19.118459,72.841763,Merwans Cake shop,19.119300,72.845418,Bakery
1,Andheri,19.118459,72.841763,Radha Krishna Veg Restaurant,19.115130,72.843060,Indian Restaurant
2,Andheri,19.118459,72.841763,Naturals,19.111204,72.837255,Ice Cream Shop
3,Andheri,19.118459,72.841763,Shawarma Factory,19.124591,72.840398,Falafel Restaurant
4,Andheri,19.118459,72.841763,Temptations,19.113767,72.841337,Ice Cream Shop


In [10]:
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")
kl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]
kl_grouped = kl_onehot.groupby(["Neighborhoods"]).mean().reset_index()
kl_grouped.head()

,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,American Restaurant,Antique Shop,Arcade,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Track,Trail,Train,Train Station,Travel & Transport,Tunnel,Vegetarian / Vegan Restaurant,Wine Bar,Wine Shop,Women's Store
0,Andheri,0.0,0.0,0.0,0.01,0.0,0.000000,0.0,0.02,0.01,...,0.0,0.0,0.0,0.000000,0.0,0.0000,0.010000,0.0,0.0,0.01
1,Anushakti Nagar,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.00,0.00,...,0.0,0.0,0.0,0.000000,0.0,0.0625,0.000000,0.0,0.0,0.00
2,Baiganwadi,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.00,0.00,...,0.0,0.0,0.0,0.000000,0.0,0.0000,0.000000,0.0,0.0,0.00
3,Bandra,0.0,0.0,0.0,0.00,0.0,0.020000,0.0,0.02,0.00,...,0.0,0.0,0.0,0.000000,0.0,0.0000,0.000000,0.0,0.0,0.00
4,Bhandup,0.0,0.0,0.0,0.00,0.0,0.043478,0.0,0.00,0.00,...,0.0,0.0,0.0,0.130435,0.0,0.0000,0.043478,0.0,0.0,0.00


In [11]:
kl_mall = kl_grouped[["Neighborhoods","Shopping Mall"]]
kl_mall.head()

,Neighborhoods,Shopping Mall
0,Andheri,0.000000
1,Anushakti Nagar,0.000000
2,Baiganwadi,0.000000
3,Bandra,0.000000
4,Bhandup,0.043478


In [12]:
kclusters = 3
kl_clustering = kl_mall.drop(["Neighborhoods"], 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)
kl_merged = kl_mall.copy()
kl_merged["Cluster Labels"] = kmeans.labels_
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged = kl_merged.join(kl_df.set_index("Neighborhood"), on="Neighborhood")
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged.head() 

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Andheri,0.0,0,19.118459,72.841763
35,Tilak Nagar (Mumbai),0.0,0,18.996160,72.852790
32,"Sion, Mumbai",0.0,0,19.043410,72.863320
31,Shil Phata,0.0,0,19.146580,73.040050
30,Seven Bungalows,0.0,0,19.131342,72.816342


In [14]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [15]:
kl_merged.loc[kl_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Andheri,0.0,0,19.118459,72.841763
35,Tilak Nagar (Mumbai),0.0,0,18.996160,72.852790
32,"Sion, Mumbai",0.0,0,19.043410,72.863320
31,Shil Phata,0.0,0,19.146580,73.040050
30,Seven Bungalows,0.0,0,19.131342,72.816342
28,Mumbra,0.0,0,19.167632,73.021408
26,Mogra Village,0.0,0,24.375900,75.954570
25,Mira Road,0.0,0,19.265674,72.870681
24,"Matharpacady, Mumbai",0.0,0,18.950694,72.827268
23,Mankhurd,0.0,0,19.048530,72.932220


In [16]:
kl_merged.loc[kl_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
36,Uttan,0.047619,1,26.86634,80.93884
19,Kanjurmarg,0.034483,1,19.13138,72.93568
38,Vikhroli,0.030000,1,19.11109,72.92781
27,Mulund,0.038961,1,19.17183,72.95565
34,Thakur village,0.034483,1,19.21020,72.87541
4,Bhandup,0.043478,1,19.14556,72.94856


In [17]:
kl_merged.loc[kl_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
37,Vashi,0.020000,2,19.084650,72.904810
39,Wadala,0.020619,2,19.017200,72.858170
5,Borivali,0.010309,2,19.229360,72.857510
29,Pestom sagar,0.011765,2,19.070640,72.902170
7,Chembur,0.010000,2,19.062180,72.902410
9,Devipada,0.010989,2,19.224690,72.866050
40,Western Suburbs (Mumbai),0.013514,2,19.197010,72.827680
12,Ghatkopar,0.012048,2,19.086523,72.909008
13,Goregaon,0.020000,2,19.164550,72.849460
33,"Sonapur, Bhandup",0.023256,2,19.163940,72.935440


In [18]:
map_clusters.save('map_clusters.html')